# OAT15 DMD analysis

In [ ]:
from os.path import join
from os import makedirs
import torch as pt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from flowtorch.analysis import DMD
from utils import normalize_frequency, add_oat_patch, spatio_temporal_correlation

plt.rcParams['figure.dpi'] = 160
plt.rc('text', usetex=True)

data = "/home/andre/Development/naca0012_shock_buffet/run/oat15"
path = "./output/oat15_analysis/dmd_analysis"

makedirs(path, exist_ok=True)

In [ ]:
CHORD = 0.15
U_INF = 238.59

times = pt.load(join(data, "oat15_tandem_times.pt"))[::20]
vertices = pt.load(join(data, "vertices_and_masks.pt"))
area = vertices["area_small"]
weight = area.sqrt().unsqueeze(-1)
x = vertices["x_small"] / CHORD
z = vertices["z_small"] / CHORD
del vertices
start_at, end_at = 101, 501 # encloses 2 cycles
dt = times[1] - times[0]

## Density - weighted and unweighted

In [ ]:
dm = pt.load(join(data, "rho_small_every10.pt"))[:, start_at:end_at:2]
dmd = DMD(dm, dt, optimal=True)
dmdw = DMD(dm*weight, dt, optimal=True)
print(dmd, dmdw)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2))

freq = normalize_frequency(dmd.frequency, CHORD, U_INF)
imp = dmd.integral_contribution[freq > 0.001]
imp /= imp.max()
freq = freq[freq > 0.001]
freqw = normalize_frequency(dmdw.frequency, CHORD, U_INF)
impw = dmdw.integral_contribution[freqw > 0.001]
impw /= impw.max()
freqw = freqw[freqw > 0.001]

markerlinew, stemlinesw, baselinew = ax.stem(freqw.numpy(), impw.numpy(), basefmt="none", linefmt=f"k-", markerfmt=f"C1o", label="weighted")
plt.setp(stemlinesw, 'linewidth', 1.0)
plt.setp(markerlinew, markersize=3)
markerline, stemlines, baseline = ax.stem(freq.numpy(), imp.numpy(), basefmt="none", linefmt=f"k-", markerfmt=f"C0x", label="unweighted")
plt.setp(stemlines, 'linewidth', 1.0)
plt.setp(markerline, markersize=3)
ax.set_xlim(0, 20)
ax.set_ylim(0, 1.05)
ax.legend()
ax.set_xlabel(r"$\tilde{f}$")
ax.set_ylabel(r"$I/I_{max}$")
plt.show()

In [ ]:
def add_mode_symmetric(ax, mode, n_levels=60):
    vmin, vmax = mode.min(), mode.max()
    vmax = max(vmin.abs(), vmax.abs())
    vmin = -vmax
    levels = pt.linspace(vmin, vmax, n_levels)
    ax.tricontourf(x, z, mode, levels=levels, vmin=vmin, vmax=vmax, cmap="seismic")

In [ ]:
n_modes = 10
top = dmd.top_modes(n_modes, integral=True, f_min=0.001)
topw = dmdw.top_modes(n_modes, integral=True, f_min=0.001)
fig, axarr = plt.subplots(n_modes, 2, figsize=(6, 1*n_modes), sharex=True, sharey=True)

for i, (mwi, mi) in enumerate(zip(topw, top)):
    add_mode_symmetric(axarr[i, 0], dmdw.modes[:, mwi].real/weight.squeeze())
    add_mode_symmetric(axarr[i, 1], dmd.modes[:, mi].real)
    axarr[i, 0].set_title(r"$\Re (\rho_w)$" + f", {normalize_frequency(dmdw.frequency[mwi], CHORD, U_INF):2.2f}")
    axarr[i, 1].set_title(r"$\Re (\rho)$" + f", {normalize_frequency(dmd.frequency[mi], CHORD, U_INF):2.2f}")
    for ax in axarr.flatten():
        ax.set_aspect("equal")
        add_oat_patch(ax)
        ax.set_ylim(-0.2, 0.5)
        ax.set_xlim(-0.1, 2.5)

for ax in axarr[:, 0]:
    ax.set_ylabel(r"$\tilde{z}$")
for ax in axarr[-1, :]:
    ax.set_xlabel(r"$\tilde{x}$")

plt.show()

## Pressure weighted and unweighted

In [ ]:
dm = pt.load(join(data, "p_small_every10.pt"))[:, start_at:end_at:2]
dmd = DMD(dm, dt, optimal=True)
dmdw = DMD(dm*weight, dt, optimal=True)
print(dmd, dmdw)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2))

freq = normalize_frequency(dmd.frequency, CHORD, U_INF)
imp = dmd.integral_contribution[freq > 0.001]
imp /= imp.max()
freq = freq[freq > 0.001]
freqw = normalize_frequency(dmdw.frequency, CHORD, U_INF)
impw = dmdw.integral_contribution[freqw > 0.001]
impw /= impw.max()
freqw = freqw[freqw > 0.001]

markerlinew, stemlinesw, baselinew = ax.stem(freqw.numpy(), impw.numpy(), basefmt="none", linefmt=f"k-", markerfmt=f"C1o", label="weighted")
plt.setp(stemlinesw, 'linewidth', 1.0)
plt.setp(markerlinew, markersize=3)
markerline, stemlines, baseline = ax.stem(freq.numpy(), imp.numpy(), basefmt="none", linefmt=f"k-", markerfmt=f"C0x", label="unweighted")
plt.setp(stemlines, 'linewidth', 1.0)
plt.setp(markerline, markersize=3)
ax.set_xlim(0, 20)
ax.set_ylim(0, 1.05)
ax.legend()
ax.set_xlabel(r"$\tilde{f}$")
ax.set_ylabel(r"$I/I_{max}$")
plt.show()

In [ ]:
n_modes = 10
top = dmd.top_modes(n_modes, integral=True, f_min=0.001)
topw = dmdw.top_modes(n_modes, integral=True, f_min=0.001)
fig, axarr = plt.subplots(n_modes, 2, figsize=(6, 1*n_modes), sharex=True, sharey=True)

for i, (mwi, mi) in enumerate(zip(topw, top)):
    add_mode_symmetric(axarr[i, 0], dmdw.modes[:, mwi].real/weight.squeeze())
    add_mode_symmetric(axarr[i, 1], dmd.modes[:, mi].real)
    axarr[i, 0].set_title(r"$\Re (p_w)$" + f", {normalize_frequency(dmdw.frequency[mwi], CHORD, U_INF):2.2f}")
    axarr[i, 1].set_title(r"$\Re (p)$" + f", {normalize_frequency(dmd.frequency[mi], CHORD, U_INF):2.2f}")
    for ax in axarr.flatten():
        ax.set_aspect("equal")
        add_oat_patch(ax)
        ax.set_ylim(-0.2, 0.5)
        ax.set_xlim(-0.1, 2.5)

for ax in axarr[:, 0]:
    ax.set_ylabel(r"$\tilde{z}$")
for ax in axarr[-1, :]:
    ax.set_xlabel(r"$\tilde{x}$")

plt.show()

## Velocity and local speed of sound

In [ ]:
vel_x = pt.load(join(data, "vel_x_small_every10.pt"))[:, start_at:end_at:2]
vel_y = pt.load(join(data, "vel_y_small_every10.pt"))[:, start_at:end_at:2]
vel_z = pt.load(join(data, "vel_z_small_every10.pt"))[:, start_at:end_at:2]
ma = pt.load(join(data, "ma_small_every10.pt"))[:, start_at:end_at:2]
speed = (vel_x**2 + vel_y**2 + vel_z**2).sqrt()
a_loc = speed / ma
kappa = pt.tensor(1.4)
scale = pt.sqrt(2.0 / (kappa * (kappa - 1.0)))
dm_axz = pt.cat((vel_x, vel_z, a_loc*scale), dim=0)
dm_xz = pt.cat((vel_x, vel_z))
del vel_x, vel_y, vel_z, ma, a_loc

In [ ]:
dmd_xz = DMD(dm_xz*weight.repeat((2, 1)), dt, optimal=True)
dmd_axz = DMD(dm_axz*weight.repeat((3, 1)), dt, optimal=True)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2))

freq_xz = normalize_frequency(dmd_xz.frequency, CHORD, U_INF)
imp_xz = dmd_xz.integral_contribution[freq_xz > 0.001]
imp_xz /= imp_xz.max()
freq_xz = freq_xz[freq_xz > 0.001]
freq_axz = normalize_frequency(dmd_axz.frequency, CHORD, U_INF)
imp_axz = dmd_axz.integral_contribution[freq_axz > 0.001]
imp_axz /= imp_axz.max()
freq_axz = freq_axz[freq_axz > 0.001]

markerline, stemlines, baseline = ax.stem(freq_xz.numpy(), imp_xz.numpy(), basefmt="none", linefmt=f"k-", markerfmt=f"C1o", label=r"$\mathbf{x}=(u_x, u_z)^T$")
plt.setp(stemlines, 'linewidth', 1.0)
plt.setp(markerline, markersize=3)
markerline, stemlines, baseline = ax.stem(freq_axz.numpy(), imp_axz.numpy(), basefmt="none", linefmt=f"k-", markerfmt=f"C0x", label=r"$\mathbf{x}=(u_x, u_z, \sqrt{2/(\gamma (\gamma -1))} a)^T$")
plt.setp(stemlines, 'linewidth', 1.0)
plt.setp(markerline, markersize=3)
ax.set_xlim(0, 20)
ax.set_ylim(0, 1.05)
ax.legend()
ax.set_xlabel(r"$\tilde{f}$")
ax.set_ylabel(r"$I/I_{max}$")
plt.show()

In [ ]:
n_modes = 10
n_points = weight.shape[0]
top_xz = dmd_xz.top_modes(n_modes, integral=True, f_min=0.001)
top_axz = dmd_axz.top_modes(n_modes, integral=True, f_min=0.001)
fig, axarr = plt.subplots(n_modes, 2, figsize=(6, 1*n_modes), sharex=True, sharey=True)

for i, (mi_xz, mi_axz) in enumerate(zip(top_xz, top_axz)):
    add_mode_symmetric(axarr[i, 0], dmd_xz.modes[:n_points, mi_xz].real/weight.squeeze())
    add_mode_symmetric(axarr[i, 1], dmd_axz.modes[:n_points, mi_axz].real/weight.squeeze())
    axarr[i, 0].set_title(r"$\Re (u_x)$" + f", {normalize_frequency(dmd_xz.frequency[mi_xz], CHORD, U_INF):2.2f}")
    axarr[i, 1].set_title(r"$\Re (u_x^a)$" + f", {normalize_frequency(dmd_axz.frequency[mi_axz], CHORD, U_INF):2.2f}")
    for ax in axarr.flatten():
        ax.set_aspect("equal")
        add_oat_patch(ax)
        ax.set_ylim(-0.2, 0.5)
        ax.set_xlim(-0.1, 2.5)

for ax in axarr[:, 0]:
    ax.set_ylabel(r"$\tilde{z}$")
for ax in axarr[-1, :]:
    ax.set_xlabel(r"$\tilde{x}$")

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2))

freq_xz = normalize_frequency(dmd_xz.frequency, CHORD, U_INF)
imp_xz = dmd_xz.integral_contribution[freq_xz > 0.001]
imp_xz /= imp_xz.max()
freq_xz = freq_xz[freq_xz > 0.001]

markerline, stemlines, baseline = ax.stem(freq_xz.numpy(), imp_xz.numpy(), basefmt="none", linefmt=f"k-", markerfmt=f"C1o", label="opt. DMD, int.")
plt.setp(stemlines, 'linewidth', 1.0)
plt.setp(markerline, markersize=4)
fb = normalize_frequency(118.5, CHORD, U_INF)
ax.axvline(fb, ls=":", c="k", label=r"$f_b$ (lift)")
ax.set_title(r"$\mathbf{x}=(u_x, u_z)^T$")
ax.set_xlim(0, 20)
ax.set_ylim(0, 1.05)
ax.set_xlabel(r"$\tilde{f}$")
ax.set_ylabel(r"$I/I_{\mathrm{max}}$")
ax.legend()
plt.savefig(join(path, "freq_vel_xz.pdf"), bbox_inches="tight")
plt.show()

In [ ]:
n_modes = 6
n_points = weight.shape[0]
top_xz = dmd_xz.top_modes(1, integral=True)
top_xz = pt.cat((top_xz, dmd_xz.top_modes(3, integral=True, f_min=0.001, f_max=400)))
top_xz = pt.cat((top_xz, dmd_xz.top_modes(2, integral=True, f_min=400)))
fig, axarr = plt.subplots(n_modes, 2, figsize=(6, 1.1*n_modes), sharex=True, sharey=True)

for i, mi_xz in enumerate(top_xz):
    add_mode_symmetric(axarr[i, 0], dmd_xz.modes[:n_points, mi_xz].real/weight.squeeze(), 100)
    add_mode_symmetric(axarr[i, 1], dmd_xz.modes[n_points:, mi_xz].real/weight.squeeze(), 100)
    axarr[i, 0].set_title(r"$\Re (u_x)$" + r", $\tilde{f} = " + "{:2.2f}$".format(normalize_frequency(dmd_xz.frequency[mi_xz], CHORD, U_INF)))
    axarr[i, 1].set_title(r"$\Re (u_z)$" + r", $\tilde{f} = " + "{:2.2f}$".format(normalize_frequency(dmd_xz.frequency[mi_xz], CHORD, U_INF)))
    for ax in axarr.flatten():
        ax.set_aspect("equal")
        add_oat_patch(ax)
        ax.set_ylim(-0.2, 0.5)
        ax.set_xlim(-0.1, 2.5)

for ax in axarr[-1, :]:
    ax.set_xlabel(r"$\tilde{x}$")
    
fig.add_subplot(111, frame_on=False)
plt.tick_params(labelcolor="none", bottom=False, left=False)
plt.ylabel(r"$\tilde{z}$")
plt.savefig(join(path, "vel_xz_modes.png"), bbox_inches="tight")
plt.show()

## Propagation speeds - vortex shedding mode

In [ ]:
top_xz

In [ ]:
top_xz_conj = [top_xz[0]]
for mi in top_xz[1:]:
    top_xz_conj.extend((dmd_xz.frequency.abs() - dmd_xz.frequency[mi].abs()).abs().topk(2, largest=False).indices)
top_xz_conj = pt.tensor(top_xz_conj, dtype=pt.int64)
top_xz_conj

In [ ]:
top_rec_err = (dmd_xz.partial_reconstruction(top_xz_conj) - dmd_xz._dm).norm()
max_rec_err = (dmd_xz.partial_reconstruction([top_xz[0]]) - dmd_xz._dm).norm()
min_rec_err = dmd_xz.reconstruction_error.norm()
top_rec_err, max_rec_err, min_rec_err

In [ ]:
(top_rec_err - min_rec_err) / (max_rec_err - min_rec_err)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
mi = top_xz[4]
phase_ux = dmd_xz.modes[:n_points, mi].angle()
vmin, vmax= -180, 180
levels = pt.linspace(vmin, vmax, 61)
tri_ux = ax.tricontourf(x, z, phase_ux*180/np.pi, levels=levels, cmap="seismic", vmin=vmin, vmax=vmax)
bar = plt.colorbar(tri_ux, ax=ax, label=r"$\mathrm{tan}^{-1}\left(\Im (u_x) / \Re (u_x)\right)$", shrink=0.8)
ax.set_aspect("equal")
add_oat_patch(ax)
ax.set_ylim(-0.5, 0.75)
ax.set_xlim(-0.1, 2.5)
ax.add_patch(
    FancyArrowPatch((1.0, 0.5), (0.5, 0.5), mutation_scale=20, fc="w")
)
ax.add_patch(
    FancyArrowPatch((1.0, -0.3), (0.5, -0.3), mutation_scale=20, fc="w")
)
ax.add_patch(
    FancyArrowPatch((1.5, 0.0), (2.0, 0.0), mutation_scale=20, fc="w")
)
ax.set_xlabel(r"$\tilde{x}$")
ax.set_ylabel(r"$\tilde{z}$")
ax.set_title(r"phase shift, $\tilde{{f}}={:2.2f}$".format(normalize_frequency(dmd_xz.frequency[mi], CHORD, U_INF)))

plt.savefig(join(path, "phase_shift.png"), bbox_inches="tight")
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 1.5))

phase_ux = dmd_xz.modes[:n_points, mi].angle()
vmin, vmax= -180, 180
levels = pt.linspace(vmin, vmax, 61)
tri_ux = ax.tricontourf(x, z, phase_ux*180/np.pi, levels=levels, cmap="seismic", vmin=vmin, vmax=vmax)
bar = plt.colorbar(tri_ux, ax=ax, label=r"$\mathrm{tan}^{-1}\left(\Im (u_x) / \Re (u_x)\right)$", shrink=0.8)
ax.set_aspect("equal")
add_oat_patch(ax)
ax.set_ylim(0.0, 0.15)
ax.set_xlim(0.5, 1.1)
ax.set_xlabel(r"$\tilde{x}$")
ax.set_ylabel(r"$\tilde{z}$")
ax.set_title(r"phase shift, $\tilde{{f}}={:2.2f}$".format(normalize_frequency(dmd_xz.frequency[mi], CHORD, U_INF)))
plt.show()

In [ ]:
np_line = 45
line = pt.stack((
    pt.linspace(0.525, 0.72, np_line),
    pt.linspace(0.3, 0.3, np_line),
)).T

In [ ]:
phase_l = []
for pi in range(line.shape[0]):
    closest = (pt.stack((x, z), dim=1) - line[pi]).norm(dim=1).argmin()
    phase_l.append(phase_ux[closest])
phase_l = pt.stack(phase_l)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2))
ax.scatter(line[:, 0], phase_l, marker="x", c="k", s=10, label=r"line sample, $\tilde{z}=0.3$")
low_1, up_1 = 0.525, 0.61
l1_range = pt.logical_and(line[:, 0] > low_1, line[:, 0] < up_1)
dm = pt.cat((phase_l[l1_range].unsqueeze(-1), line[l1_range, 0].unsqueeze(-1)), dim=1).T
dm_mean = dm.mean(dim=1)
U, s, VH = pt.linalg.svd(dm - dm_mean.unsqueeze(-1), full_matrices=False)
dphi_dx = U[0, 0] / U[1, 0]
u_rel_1 = -2*np.pi*dmd_xz.frequency[mi] / dphi_dx / U_INF * CHORD
phi_1_rec = dm_mean[0] + dphi_dx * (line[l1_range, 0] - dm_mean[1])
ax.plot(line[l1_range, 0], phi_1_rec, ls="--", c="C0", label="linear fit")
ax.text(0.55, -0.5, r"$U_p=" + f"{u_rel_1:2.2f}" + "U_\infty$", rotation=35)

low_2, up_2 = 0.613, 0.703
l2_range = pt.logical_and(line[:, 0] > low_2, line[:, 0] < up_2)
dm = pt.cat((phase_l[l2_range].unsqueeze(-1), line[l2_range, 0].unsqueeze(-1)), dim=1).T
dm_mean = dm.mean(dim=1)
U, s, VH = pt.linalg.svd(dm - dm_mean.unsqueeze(-1), full_matrices=False)
dphi_dx = U[0, 0] / U[1, 0]
u_rel_2 = -2*np.pi*dmd_xz.frequency[mi] / dphi_dx / U_INF * CHORD
phi_2_rec = dm_mean[0] + dphi_dx * (line[l2_range, 0] - dm_mean[1])
ax.plot(line[l2_range, 0], phi_2_rec, ls="--", c="C0")
ax.text(0.635, -0.5, r"$U_p=" + f"{u_rel_2:2.2f}" + "U_\infty$", rotation=35)
ax.set_xlabel(r"$\tilde{x}$")
ax.set_ylabel(r"$\mathrm{tan}^{-1}\left(\Im (u_x) / \Re (u_x)\right)$")
ax.legend()
#ax.axvline(0.5275, ls=":", c="k")
#ax.axvline(0.6115, ls=":", c="k")
#ax.text(0.55, -2, r"$\Delta x \approx {:2.3f}c$".format(0.6115-0.5275))
plt.savefig(join(path, "phase_speed.pdf"), bbox_inches="tight")
plt.show()

## Propagation speeds - buffet mode (based on pressure)

In [ ]:
dm = pt.load(join(data, "p_small_every10.pt"))[:, start_at:end_at:2]
dmdw = DMD(dm*weight, dt, optimal=True)

In [ ]:
topw = dmdw.top_modes(10, integral=True, f_min=0.001)
topw

In [ ]:
normalize_frequency(dmdw.frequency[topw], CHORD, U_INF)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
mi = topw[0]
phase_p = dmdw.modes[:, mi].angle()
vmin, vmax= -180, 180
levels = pt.linspace(vmin, vmax, 61)
tri_p = ax.tricontourf(x, z, phase_p*180/np.pi, levels=levels, cmap="seismic", vmin=vmin, vmax=vmax)
bar = plt.colorbar(tri_p, ax=ax, label=r"$\mathrm{tan}^{-1}\left(\Im (p) / \Re (p)\right)$", shrink=0.8)
ax.set_aspect("equal")
add_oat_patch(ax)
ax.set_ylim(-0.5, 0.75)
ax.set_xlim(-0.1, 2.5)
ax.set_xlabel(r"$\tilde{x}$")
ax.set_ylabel(r"$\tilde{z}$")
ax.set_title(r"phase shift, $\tilde{{f}}={:2.2f}$".format(normalize_frequency(dmdw.frequency[mi], CHORD, U_INF)))

plt.savefig(join(path, "phase_shift_buffet.png"), bbox_inches="tight")
plt.show()

In [ ]:
np_line_1 = 20
line_1 = pt.stack((
    pt.linspace(0.6, 0.9, np_line_1),
    pt.linspace(0.078, 0.033, np_line_1),
)).T
np_line_2 = 10
line_2 = pt.stack((
    pt.linspace(0.88, 1.0, np_line_2),
    pt.linspace(-0.008, -0.015, np_line_2),
)).T
np_line_3 = 15
line_3 = pt.stack((
    pt.linspace(0.3, 0.8, np_line_3),
    pt.linspace(-0.08, -0.015, np_line_3),
)).T

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3))

vmin, vmax= -180, 180
levels = pt.linspace(vmin, vmax, 61)
tri_p = ax.tricontourf(x, z, phase_p*180/np.pi, levels=levels, cmap="seismic", vmin=vmin, vmax=vmax)
bar = plt.colorbar(tri_p, ax=ax, label=r"$\mathrm{tan}^{-1}\left(\Im (p) / \Re (p)\right)$", shrink=0.8)
ax.set_aspect("equal")
add_oat_patch(ax)
ax.set_ylim(-0.15, 0.4)
ax.set_xlim(0.0, 1.1)
ax.plot(line_1[:, 0], line_1[:, 1], lw=1, c="k", ls="--")
ax.plot(line_2[:, 0], line_2[:, 1], lw=1, c="k", ls="--")
ax.plot(line_3[:, 0], line_3[:, 1], lw=1, c="k", ls="--")
ax.set_xlabel(r"$\tilde{x}$")
ax.set_ylabel(r"$\tilde{z}$")
ax.text(0.45, -0.1, r"$-0.25 U_\infty$", rotation=7)
ax.text(0.7, 0.055, r"$0.070 U_\infty$", rotation=-10)
ax.text(0.85, -0.06, r"$-0.17 U_\infty$", rotation=-5)
ax.set_title(r"phase shift, $\tilde{{f}}={:2.2f}$".format(normalize_frequency(dmdw.frequency[mi], CHORD, U_INF)))
plt.savefig(join(path, "phase_shift_buffet_enlarged.png"), bbox_inches="tight")
plt.show()

In [ ]:
phase_l1 = []
for pi in range(line_1.shape[0]):
    closest = (pt.stack((x, z), dim=1) - line_1[pi]).norm(dim=1).argmin()
    phase_l1.append(phase_p[closest])
phase_l1 = pt.stack(phase_l1)
phase_l2 = []
for pi in range(line_2.shape[0]):
    closest = (pt.stack((x, z), dim=1) - line_2[pi]).norm(dim=1).argmin()
    phase_l2.append(phase_p[closest])
phase_l2 = pt.stack(phase_l2)
phase_l3 = []
for pi in range(line_3.shape[0]):
    closest = (pt.stack((x, z), dim=1) - line_3[pi]).norm(dim=1).argmin()
    phase_l3.append(phase_p[closest])
phase_l3 = pt.stack(phase_l3)

In [ ]:
fig, axarr = plt.subplots(1, 3, figsize=(6, 2), sharey=True, sharex=False)

# line 1 - boundary layer suction side
ds_1 = (line_1 - line_1[0, :]).norm(dim=1)
axarr[0].scatter(ds_1, phase_l1, marker="x", c="k", s=10)
low_1, up_1 = 0.0, 0.4
l1_range = pt.logical_and(ds_1 > low_1, ds_1 < up_1)
dm = pt.cat((phase_l1[l1_range].unsqueeze(-1), line_1[l1_range, 0].unsqueeze(-1)), dim=1).T
dm_mean = dm.mean(dim=1)
U, s, VH = pt.linalg.svd(dm - dm_mean.unsqueeze(-1), full_matrices=False)
dphi_ds_1 = U[0, 0] / U[1, 0]
u_rel_1 = -2*np.pi*dmdw.frequency[mi] / dphi_ds_1 / U_INF * CHORD
phi_1_rec = dm_mean[0] + dphi_ds_1 * (line_1[l1_range, 0] - dm_mean[1])
axarr[0].plot(ds_1[l1_range], phi_1_rec, ls="--", c="C0", label="linear fit")
axarr[0].set_title(r"$U_p=" + f"{u_rel_1:2.3f}" + "U_\infty$")

# line 2 - pressure side towards trailing edge
ds_2 = (line_2 - line_2[0, :]).norm(dim=1)
axarr[1].scatter(ds_2, phase_l2, marker="x", c="k", s=10)
low_2, up_2 = 0.0, 0.15
l2_range = pt.logical_and(ds_2 > low_2, ds_2 < up_2)
dm = pt.cat((phase_l2[l2_range].unsqueeze(-1), line_2[l2_range, 0].unsqueeze(-1)), dim=1).T
dm_mean = dm.mean(dim=1)
U, s, VH = pt.linalg.svd(dm - dm_mean.unsqueeze(-1), full_matrices=False)
dphi_ds_2 = U[0, 0] / U[1, 0]
u_rel_2 = -2*np.pi*dmdw.frequency[mi] / dphi_ds_2 / U_INF * CHORD
phi_2_rec = dm_mean[0] + dphi_ds_2 * (line_2[l2_range, 0] - dm_mean[1])
axarr[1].plot(ds_2[l2_range], phi_2_rec, ls="--", c="C0", label="linear fit")
axarr[1].set_title(r"$U_p=" + f"{u_rel_2:2.2f}" + "U_\infty$")

# line 3 - pressure side center
ds_3 = (line_3 - line_3[0, :]).norm(dim=1)
axarr[2].scatter(ds_3, phase_l3, marker="x", c="k", s=10)
low_3, up_3 = 0.0, 0.5
l3_range = pt.logical_and(ds_3 > low_3, ds_3 < up_3)
dm = pt.cat((phase_l3[l3_range].unsqueeze(-1), line_3[l3_range, 0].unsqueeze(-1)), dim=1).T
dm_mean = dm.mean(dim=1)
U, s, VH = pt.linalg.svd(dm - dm_mean.unsqueeze(-1), full_matrices=False)
dphi_ds_3 = U[0, 0] / U[1, 0]
u_rel_3 = -2*np.pi*dmdw.frequency[mi] / dphi_ds_3 / U_INF * CHORD
phi_3_rec = dm_mean[0] + dphi_ds_3 * (line_3[l3_range, 0] - dm_mean[1])
axarr[2].plot(ds_3[l3_range], phi_3_rec, ls="--", c="C0", label="linear fit")
axarr[2].set_title(r"$U_p=" + f"{u_rel_3:2.2f}" + "U_\infty$")
for ax in axarr.flatten():
    ax.set_xlabel(r"$\tilde{s}$")
axarr[0].set_ylabel(r"$\mathrm{tan}^{-1}\left(\Im (p) / \Re (p)\right)$")
plt.savefig(join(path, "phase_speed_buffet.pdf"), bbox_inches="tight")
plt.show()

## UDMD buffet mode - rank dependency

In [ ]:
ranks = [25, 50, 100]
n_points = weight.shape[0]
top_mode_dmd, top_mode_udmd = [], []
for r in ranks:
    dmd = DMD(dm_xz*weight.repeat((2, 1)), dt, rank=r, optimal=True)
    top = dmd.top_modes(1, integral=True, f_min=1)
    top_mode_dmd.append(dmd.modes[:n_points, top[0]].real)
    dmd = DMD(dm_xz*weight.repeat((2, 1)), dt, rank=r, optimal=True, unitary=True)
    top = dmd.top_modes(1, integral=True, f_min=1)
    top_mode_udmd.append(dmd.modes[:n_points, top[0]].real)

In [ ]:
fig, axarr = plt.subplots(2, 3, figsize=(6, 1.5), sharex=True, sharey=True)

for i, r in enumerate(ranks):
    add_mode_symmetric(axarr[0, i], top_mode_udmd[i] / weight.squeeze())
    add_mode_symmetric(axarr[1, i], top_mode_dmd[i] / weight.squeeze())
    axarr[0, i].set_title(r"$\Re (u_x)$, $r={:d}$".format(r))
    axarr[1, i].set_xlabel(r"$\tilde{x}$")
    for ax in axarr[:, i]:
        ax.set_aspect("equal")
        add_oat_patch(ax)
        ax.set_ylim(-0.2, 0.5)
        ax.set_xlim(-0.1, 2.5)

axarr[0, 0].set_ylabel(r"$\tilde{z}$")
axarr[1, 0].set_ylabel(r"$\tilde{z}$")
axarr[0, 2].text(2.6, 0.2, "UDMD", rotation=90, va="center")
axarr[1, 2].text(2.6, 0.2, "DMD", rotation=90, va="center")
plt.savefig(join(path, "top_mode_udmd_dmd.png"), bbox_inches="tight")
plt.show()